In [22]:
from common import *
import warnings
warnings.filterwarnings('ignore')
import operator

In [23]:
_TEST_ = False

In [24]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

# -=#=-=#=-=#=-=#=-=#=- Features selection logic -=#=-=#=-=#=-=#=-=#=-=#=-

### -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- Abstract Function datatype -=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-

In [25]:
from types import FunctionType

##### item checker

In [26]:
def item_check(a):
    if not isinstance(a, tuple):
        a = (a,)
    return a

#### Function super useful datatype

In [27]:
class Function(type):
    def __new__(self, *args, **kw):
        noattach = '_noattach_' in kw
        if len(args) == 1:
            if isinstance(args[0], __class__):
                return args[0]
            assert callable(args[0])
            if 'name' in kw:
                name = kw['name']
            else:
                name = args[0].__name__
            parents = ()
            attrs = {'__call__': args[0]}
            noattach = True
        else:
            name, parents, attrs = args
        for k, v in list(dict.items(attrs)):
            if k == '__getitem__':
                assert isinstance(v, FunctionType), "just for easier work on our side. it is a function(self, index)"
                if isinstance(v(self, 0), FunctionType): # if getitem returns a function
                    old_v = v
                    v = lambda self, a: __class__(old_v(self, item_check(a)))
            if not isinstance(v, classmethod) and callable(v) and not isinstance(v, __class__) and not noattach:
                attrs[k] = classmethod(v)
            
        return super().__new__(self, name, parents, attrs)
    
    def __init__(self, *args, **kw):
        #SEE('init', args, kw)
        if len(args) == 1:
            return
        name, parents, attrs = args
        for k in attrs:
            v = getattr(self, k)
            #SEE('init ->', k, v)
            if k[:1] != '_' and callable(v):
                setattr(self, k, __class__(v, _noattach_=True, name=k))
        
    def __call__(self, *a, **k):
        assert '__call__' in self.__dict__, f'__call__ not defined for Function object with name "{self.__name__}"'
        return self.__call__(*a, **k)
        
    def __getitem__(self, *a, **k):
        assert '__getitem__' in self.__dict__, f'__getitem__ not defined for Function object with name "{self.__name__}"'
        return self.__getitem__(*a, **k)
    
    __invert__ = lambda f: __class__(lambda *a, **k: ~f(*a,**k))
    __and__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) & g(*a,**k))
    __or__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) | g(*a,**k))
    __xor__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) ^ g(*a,**k))

In [28]:
# testing
if _TEST_:
    class What(metaclass=Function):
        def __call__(self, x, y):
            return np.sqrt(x**2 + y**2)
        def hi(self, *s):
            pass
            #SEE('hi, ' + s[0])
        __getitem__ = lambda self, a: lambda arg: (SEE('this is getitem', arg, arg*a), 'answer')[1]
    What.hi('u')
    What.hi#.__call__
    /SEE What[5].__class__
    /SEE What[5](6)

##### item getter

In [29]:
class Itemgetter():
    def __init__(self, func):
        self._func = Function(func)
    def __getitem__(self, a):
        return self._func(item_check(a))

In [30]:
# testing
if _TEST_:
    guy = Itemgetter(lambda *a: SEE(*a))
    guy[5.7,8]

In [31]:
# testing yo!
if _TEST_:
    class Yo():
        def __call__(self, x):
            SEE(x, '!')
    yo = Yo()
    yo.__call__ = lambda x: SEE('fuck', x)
    yo(5)

#### Function query language implementation

In [109]:
def scan(s):
    s = s.replace(' ','').replace('\n','').replace('\t','').replace('\r','') + '\0'
    i0, i = 0, 0
    while i0 < len(s)-1 and i < len(s):
        c = s[i]
        if 'a' <= c <= 'z' or 'A' <= c <= 'Z' or '0' <= c <= '9' or c == '_':
            i += 1
            continue
        elif c == '\0':
            yield s[i0:i]
            break
        elif c in '.~&|^()':
            if i0 < i:
                yield s[i0:i]
            yield c
            i0 = i = i+1
        elif c in '[{<':
            if i0 < i:
                yield s[i0:i]
            i0 = i
            op = c
            cl = {'[':']','{':'}','<':'>'}[op]
            opened = 1
            while opened > 0:
                i += 1
                opened += 1 if s[i] == op else -1 if s[i] == cl else 0
            i += 1
            if op != '<': # if this is not a comment
                yield s[i0:i]
            i0 = i
        else:
            raise AssertionError('bad syntax in input to scanner')

            
Token = namedtuple('Token', 'type value')

def tokenize(s):
    prevIsField = False
    for w in s:
        a = w[0]
        if w == '.':
            yield Token('self', '')
        elif '0' <= a <= '9':
            yield Token('literal', w)
        elif 'a' <= a <= 'z' or 'A' <= a <= 'Z' or a in '_':
            yield Token('field', w)
            prevIsField = True
            continue
        elif w in ['&','^','|']:
            yield Token('op2', w)
        elif w in ['~']:
            yield Token('op1', w)
        elif w in ['(']:
            yield Token('_group', w)
        elif w in [')']:
            yield Token('group_', w)
        elif a == '{':
            assert prevIsField, '{} section must have been preceded by a field name'
            yield Token('op2', 'query')
            yield Token('raw', w)
        elif a == '[':
            if not prevIsField:
                yield Token('self', '')
            yield Token('op2', 'getitem')
            yield Token('raw', w)
        else:
            assert False, f'invalid word encountered in tokenizer input: {w}'
        prevIsField = False
        
class TokenInfo(O()):
    self = 'self, the owning Function instance'
    field = 'field name of Function instance'
    op2 = 'binary operator (look in the map)'
    op1 = 'unary operator (look in the map)'
    _group = "open grouping (just '(')"
    group_ = "close grouping (just ')')"
    raw = "raw data to be fed to operator"
    literal = "literal value. either '0' or '1'"


def normalize(s):
    def reduce_op2(s):
        '''removes repeated binary operators or binary operators missing left/right argument'''
        prev = None
        hardprev = None
        for t in s:
            if (prev is not None and prev.type=='_group' or hardprev is None) and t.type=='op2':
                #^ e.g. "(| A | B)" or "& B", remove useless first binop
                pass # don't set prev=t and defer yielding prev (
            elif hardprev is not None and hardprev.type=='op2' and t.type=='op2': # repeat op2
                assert hardprev.value==t.value, 'repeated op2 must be the same'
                # don't set prev=t, no non-redunt piece of resulting query has been seen
            elif prev is not None and prev.type=='op2' and t.type=='group_':
                # don't yield prev
                prev = t
            elif prev is None:
                # attempt to yield `prev` next loop iter
                prev = t
            else:
                yield prev
                prev = t
            hardprev = t
        if prev is not None and prev.type!='op2':
            yield prev
    def reduce_empty(s):
        '''replace empty groups () with 0'''
        prev = None
        for t in s:
            if prev is None:
                prev = t
            elif prev.type=='_group' and t.type=='group_':
                prev = Token('literal', '0')
            else:
                yield prev
                prev = t
        yield prev
    yield from reduce_empty(reduce_op2(s))
    
def parse(s):
    stack = []
    for t in s:
        if t.type in ['field', 'self', 'raw', 'literal']:
            yield t
        elif t.type in ['op2', 'op1']:
            while stack and stack[-1].type in ['op2', 'op1'] and parse.order[t.value] >= parse.order[stack[-1].value]:
                yield stack.pop()
            stack.append(t)
        elif t.type == '_group':
            stack.append(t)
        elif t.type == 'group_':
            while True:
                if not stack:
                    raise AssertionError('too many close parentheses')
                x = stack.pop()
                if x.type == '_group':
                    break
                else:
                    assert x.type in ['op2', 'op1'], 'internal error'
                    yield x
    while stack:
        x = stack.pop()
        if x.type == '_group':
            raise AssertionError('too many open parentheses')
        assert x.type in ['op2', 'op1'], 'internal error'
        yield x
parse.order = {'query': 0, 'getitem': 0, '~': 1, '&': 2, '^': 3, '|': 4}

    
def query(self, qs):
    revpol = list(parse(normalize(tokenize(scan(qs)))))
    stack = []
    BUG.stack = stack
    for t in revpol:
        if t.type == 'self':
            stack.append(self)
        elif t.type == 'field':
            stack.append(getattr(self, t.value))
        elif t.type == 'literal':
            if t.value == '0':
                stack.append(Function(lambda*a,**k:False))
            elif t.value == '1':
                stack.append(Function(lambda*a,**k:True))
            else:
                assert False, f'unsupported literal value found, token={t}'
        elif t.type == 'raw':
            stack.append(t.value)
        elif t.type == 'op1':
            a = stack.pop()
            x = query.opTbl[t.value](a)
            stack.append(x)
        elif t.type == 'op2':
            b, a = stack.pop(), stack.pop()
            x = query.opTbl[t.value](a, b)
            stack.append(x)
    return stack.pop()

class GetItemHelper(metaclass=Singleton):
    def __getitem__(self, a):
        return a

query.opTbl = {
    '~': operator.__invert__,
    '&': operator.__and__,
    '^': operator.__xor__,
    '|': operator.__or__,
    'query': lambda f, q: f.query(q[1:-1]), #TODO can only go one level deep
    'getitem': lambda f, a: f[eval(f'GetItemHelper()[{a[1:-1]}]')]
}

Function.query = query

In [110]:
# testing function query
if _TEST_:
    qs = 'what2{f_ck} & no_[5,62,6:6] <whatevercomment> |  [9] & ~(ho_10|[3,5:9])'
    scanned = list(scan(qs))
    assert (scanned == ['what2','{f_ck}','&','no_','[5,62,6:6]','|','[9]','&','~','(','ho_10','|','[3,5:9]',')']), (
            'Unit test fail for scanner')    
    tokens = list(tokenize(scanned)) #TODO make unit test
    parsed = list(parse(tokens)) # TODO make unit test
    SEE(qs)
    SEE('-'*99)
    SEE(scanned)
    SEE('-'*99)
    for line in tokens:
        SEE(line)
    SEE('-'*99)
    for line in parsed:
        SEE(line)

In [111]:
def stringify(self, name=None):
    name = self.__name__ if name is None else name
    cond = lambda field: not(field=='query' or field[:1]=='_' and field!='__getitem__')
    if name == '__getitem__':
        yield '[]'
        return
    yield name
    if not any(cond(f) for f in dir(self)):
        return
    yield '{'
    for field in self.__dict__:
        if not cond(field):
            continue
        value = getattr(self, field)
        yield from stringify(getattr(self, field), field)
        yield ','
    yield '}'
Function.stringify = stringify

## .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!. The actual Features subsets definitions!!!! .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

In [373]:
def time_slice(e):
    start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
    assert start >= 0 and stop >= 0
    return max(start, stop), min(start, stop)

def digit_check(x, s):
    '''checks `s in x` but what follows first occurence of `s` isn't a digit char'''
    try:
        i = x.index(s)
        return i+len(s)>=len(x) or x[i+len(s)] in '_ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz{'
    except ValueError:
        return False
    
def i2s(e):
    if isinstance(e, slice):
        lrg, sml = time_slice(e)
    elif isinstance(e, int):
        lrg, sml = e, 0
    else:
        assert False, 'unexpected input type in function `i2s`'
    if sml == 0:
        return str(lrg)
    elif lrg % 5 == sml % 5 == 0:
        return f'({lrg}-{sml})'
    elif lrg - sml == 1:
        return f'{{{sml}}}'
    else:
        return f'{{{sml}/{lrg}}}'

def gen_getitem(*words):
    return lambda f, a: lambda x: any(digit_check(x, word+i2s(e)) for e in item_check(a) for word in words)

def index_check(x, e):
    if isinstance(e, slice):
        fst, snd = e.start, e.stop
        return x[::-1].find(f'{{{fst}/{snd}}}'[::-1]) == 0
    elif e == 0:
        return x[-1] != '}'
    elif isinstance(e, int):
        return x[::-1].find(f'{{{e}}}'[::-1]) == 0
    else:
        assert False, 'wrong type of argument `e` to function index_check'

def gen_getindex():
    return lambda f, a: lambda x: any(index_check(x, e) for e in item_check(a))


def weirdmix(x, e):
    a = '(it,af,it{1},aoo,acc)'
    b = '(it{1},af{1},it{2},aoo{1},acc{1})'
    i = [None,'{0/1}','{1/2}','{0/2}','{0/3}','{1/3}','{2/3}','{0/4}','{1/4}','{2/4}']
    num = e % 10
    s = a if e < 10 else b
    return x == s+i[num]
    


class FFF(metaclass=Function):
    # BIG NOTE item_check isn't necessary for the __getitem__ definitions anymore, it's done automatically by metaclass
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ super basic ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class AssetEnc(metaclass=Function):
        __call__ = lambda f, x: 'assetCode' in x or 'assetName' in x
        Code = lambda f, x: x  == 'assetCodeId'
        
        class RandMap(metaclass=Function):
            __call__ = lambda f, x: 'randMap32' in x
            squared = Itemgetter(lambda a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e**2))[::-1]) == 0 for e in item_check(a)))
            __getitem__ = (lambda f, a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e))[::-1]) == 0 for e in item_check(a)))
        
        WTFF = lambda f, x: 'WTFF' in x
        InUni = lambda f, x: 'inUniCount' in x
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Since and MinMax pools ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    class Since(metaclass=Function):
        __call__ = lambda f, x: 'Since' in x
        __getitem__ = gen_getitem('Since')
        class index(metaclass=Function):
            __call__ = lambda f, x: x[-1] == '}'
            __getitem__ = gen_getindex()
        Max = lambda f, x: 'max' in x or 'Max' in x
        Min = lambda f, x: 'min' in x or 'Min' in x
        
    class FracRec(metaclass=Function):
        __call__ = lambda f, x: 'fracRecordedDays' in x
        __getitem__ = gen_getitem('fracRecordedDaysSince')
        
    class Volatility(metaclass=Function):
        __call__ = lambda f, x: 'olatility' in x
        __getitem__ = gen_getitem('olatility')
    
    class Drawdown(metaclass=Function):
        __call__ = lambda f, x: 'rawdown' in x
        __getitem__ = gen_getitem('rawdown')
    
    class Drawup(metaclass=Function):
        __call__ = lambda f, x: 'rawup' in x
        __getitem__ = gen_getitem('rawup')
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Stock day by day values ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class index(metaclass=Function):
        __call__ = lambda f, x: x[-1] == '}'
        __getitem__ = gen_getindex()
    
    class FaceValue(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close']
        Open = lambda f, x: x == 'open'
        Close = lambda f, x: x == 'close'
        Volume = lambda f, x: x == 'volume'
        
    class VP(metaclass=Function):
        __call__ = lambda f, x: (x[:2] == 'vp' or x[:4] == 'm_vp') and 'dd' not in x
        __getitem__ = gen_getitem('vp')
        
    class Return(metaclass=Function):
        pure = lambda f, x: all(b not in x for b in
                                ['inSince','axSince','rawdown','rawup','ecordedDays','ssetCode','olume'])
        __call__ = lambda f, x: any(ww in x for ww in ['oo','cc','dd','(oo-aoo)','(cc-acc)','(oo-cc)','(aoo-acc)'])
        class oo(metaclass=Function):
            __call__ = lambda f, x: 'oo' in x and not any(s in x for s in ['aoo','-oo','oo-',',oo','oo,'])
            __getitem__ = gen_getitem('oo')
        class aoo(metaclass=Function):
            __call__ = lambda f, x: 'aoo' in x and not any(s in x for s in ['-aoo','aoo-',',aoo','aoo,'])
            __getitem__ = gen_getitem('aoo')
        class doo(metaclass=Function):
            __call__ = lambda f, x: '(oo-aoo)' in x
            __getitem__ = gen_getitem('(oo-aoo)')
            
        class cc(metaclass=Function):
            __call__ = lambda f, x: 'cc' in x and not any(['acc' in x, '-cc' in x, 'cc-' in x, ',cc' in x, 'cc,' in x])
            __getitem__ = gen_getitem('cc')
        class acc(metaclass=Function):
            __call__ = lambda f, x: 'acc' in x and not any(['-acc' in x, 'acc-' in x, ',acc' in x, 'acc,' in x])
            __getitem__ = gen_getitem('acc')
        class dcc(metaclass=Function):
            __call__ = lambda f, x: '(cc-acc)' in x
            __getitem__ = gen_getitem('(cc-acc)')
            
        class doc(metaclass=Function):
            __call__ = lambda f, x: '(oo-cc)' in x
            __getitem__ = gen_getitem('(oo-cc)')
        class daoc(metaclass=Function):
            __call__ = lambda f, x: '(aoo-acc)' in x
            __getitem__ = gen_getitem('(aoo-acc)')
            
        class dd(metaclass=Function):
            __call__ = lambda f, x: 'dd' in x
            __getitem__ = gen_getitem('dd')
        class vp1dd(metaclass=Function):
            __call__ = lambda f, x: 'vp1dd' in x
            __getitem__ = gen_getitem('vp1dd')
        class vp5dd(metaclass=Function):
            __call__ = lambda f, x: 'vp5dd' in x
            __getitem__ = gen_getitem('vp5dd')
        class vp5dd(metaclass=Function):
            __call__ = lambda f, x: 'vp10dd' in x
            __getitem__ = gen_getitem('vp10dd')
            
        class Volatility(metaclass=Function):
            __call__ = lambda f, x: 'olatility' in x
            __getitem__ = gen_getitem('olatility')
        
        __getitem__ = gen_getitem('oo','aoo','(oo-aoo)','vp1dd','vp5dd','volatility')
        class index(metaclass=Function):
            __call__ = lambda f, x: x[-1] == '}'
            __getitem__ = gen_getindex()
        class digit(metaclass=Function):
            __call__ = lambda f, x: x[-1] in '0123456789)'
            
        mix = Itemgetter(lambda a: lambda x: any(weirdmix(x, e) for e in item_check(a)))
        rr = lambda f, x: 'rr' in x
        it = lambda f, x: x[:3] in ['it', 'it{']
        af = lambda f, x: x[:3] in ['af', 'af{']
        TEN = lambda f, x: any(s+'TEN' in x for s in ['oo','cc','aoo','acc'])
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ MARKET ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class Time(metaclass=Function):
        __call__ = lambda f, x: x in ['dayOfYear', 'dayOfWeek']
        long = lambda f, x: x == 'dayOfYear'
        short = lambda f, x: x == 'dayOfWeek'
        
    class Market(metaclass=Function):
        __call__ = lambda f, x: 'm_' in x
        Return = lambda f, x: 'oo' in x
        VP = lambda f, x: 'vp' in x
        Volatility = lambda f, x: 'olatility' in x
        __getitem__ = gen_getitem('oo','dd','olatility')

        class Weight(metaclass=Function):
            __call__ = lambda f, x: 'wVP' in x
    

In [374]:
import json
#json.dump(list(F.columns), open('feats.json', 'w'))
feats = json.load(open(top_dir/'work/feats.json', 'r'))

In [419]:
assert ''.join(list(FFF.stringify())) # super lazy no-error check

In [468]:
fffTemplate = '''
    <>~Market & (
        <>Return{.&pure&~digit&~TEN} & ~Since & Return{.& = .5
            <>(oo?.6|cc?.6|aoo?.3|acc?.3|doo?.3|dcc?.3|doc?.3|daoc?.3|vp1dd?.5)
            & index[0?.7,1?.7,2?.7, 0:1?.7,1:2?.7,0:2?.7] @index
          }
        | Return{.&pure&~index} & ~Since & Return{.& = .5
            <>(oo?.7|aoo?.5|doo?.3|vp1dd?.4|vp5dd?.6|Volatility?.4)
            & [5:?.5,10:?.7,15:?.6,10:5?.5,15:10?.5,20:15?.5,15:5?.7,20:10?.7,20:5?.6] @med
          }
        | ( = .5
            <>Return{(oo{.&[1:]}?.8|aoo[1:]?.2|doo[1:]?.2|oo{.&[10:]}?.6|aoo[10:]?.2|doo[10:]?.2)} @smooth
            & (
                <> ~Drawdown&~Drawup&~Since{Min}&~Since{Max} =.5
                | Drawdown[1:?.5, 5:?.4,10:?.6,20:?.5, 10:5?.4,20:10?.6] @medlong
                | Since{Max&index[0?.7,5?.2,10?.2,20?.2]} =.5 @medlong
                | Drawup[1:?.2, 5:?.1,10:?.2,20:?.2, 10:5?.1,20:10?.2] =.3 @medlong
                | Since{Min&index[0?.6,5?.2,10?.3,20?.2]} @medlong
              )
            & Since[21:?.5, 62:?.7, 125:?.3, 250:?.3] @long
          )
        | ( = .4
            <>Return{dd} & VP[1:?.3, 5:?.5, 10:?.5] @smooth
            & (
                <> ~Drawdown&~Drawup&~Since{Min}&~Since{Max} =.7
                | Drawdown[1:?.35, 5:?.2,10:?.2,20:?.2, 10:5?.2,20:10?.35] =.5 @medlong
                | Since{Max & index[0?.7,5?.2,10?.35,20?.2]} @medlong
                | Drawup[1?.5, 5:?.2,10:?.3,20:?.5, 10:5?.2,20:10?.3] =.6 @medlong
                | Since{Min & index[0?.7,5?.2,10?.35,20?.25]} @medlong
              )
            & Since[21:?.15, 62:?.25, 125:?.5, 250:?.6] @long
          )
        | ( = .4
            <> Volatility[20:?.5, 60:?.2] =.7 @long
            | Volatility[10:?.6, 20:?.6]
            & (
                <>Drawdown[1:?.4, 5:?.3,10:?.2,20:?.2, 10:5?.3,20:10?.2] =.4 @medlong
                | Since{Max & index[0?.3,5?.3,10?.5,20?.1]} =.8 @medlong
                | Drawup[1:?.4, 5:?.3,10:?.2,20:?.2, 10:5?.3,20:10?.2] =.4 @medlong
                | Since{Min & index[0?.3,5?.3,10?.5,20?.1]} =.8 @medlong
              )
            & Since[21:?.1, 62:?.2, 125:?.5, 250:?.5] @long
          )
      )
    | Market{.& = .5
        <>(~Weight?.5 | Weight?.5)
        & (Return?.7 | VP?.7 | Volatility?.5)
        & [1:?.4,5:?.3,10:?.4,20:?.5,60:?.4] @short @med @long
      }
    | = .3
        <>Return{pure&~dd} & VP[1:?.2,5:?.1,10:?.2,20:?.1,60:?.1] =.8 @short @med @long
        | FracRec[21:?.5,62:?.3,125:?.1,250:?.3] =.8 @long
    | = .25
        <>AssetEnc{InUni} =.7
        | AssetEnc{Code} =.3
    | = .1
        <>FaceValue{Volume?.6 | Open?.5 | Close?.5} = .7
        | Return{mix[1?.5,2?.5,3?.5,4?.5,5?.5,6?.5,7?.5,8?.5,9?.5,11?.5,12?.5,13?.5,14?.5,15?.5,16?.5,17?.5,18?.5,19?.5]}=.6
        | Return{rr?.5 | af?.5 | it?.5} =.8
    | = .4
        Time{long?.9 | short?.5}
'''

In [477]:
def fill_template(lines, samp, edit):
    '''Parameters - lines: iterable of lines of the template
                    samp: function float([0,1]) -> float, >0 means choose it, <0 means don't choose it
                    edit: object mapping name -> TODO
    Yield the processed lines that should join to make a valid query string
    '''
    import re
    whitespace = re.compile(r'\s')
    Form = namedtuple('Form', 're op sep cl')
    forms = [
        Form(re.compile(r'\[.*\S+.*,.*\S+.*\]'), '[', ',', ']'),
        Form(re.compile(r'\(.*\S+.*\|.*\S+.*\)'), '(', '|', ')'),
        Form(re.compile(r'\{.*\S+.*\|.*\S+.*\}'), '{', '|', '}'),
    ]
    
    del_indent = None
    
    for lnum, line in enumerate(lines):
        if line.strip() == '':
            continue
        
        indent = 0
        while whitespace.match(line[indent]):
            assert line[indent] == ' ', f'indents must be space | {lnum}: {line}'
            indent += 1
        if del_indent is not None and indent > del_indent: # still deleting a block
            continue
        del_indent = None
        
        if '=' not in line and '?' not in line:
            assert '@' not in line
            yield line
            continue
        elif '=' in line:
            l0 = line.index('=')
            l1 = line.index('@') if '@' in line else None
            
            x = float(line[l0+1:l1])
            y = samp(x)
            if y <= 0: # delete whole block
                del_indent = indent
                continue
                
            line = line[:l0]
        
        # at this point the line is definitely in, we just need to process it
        if '@' in line:
            line = line[:line.index('@')] # we're just not gonna deal with '@' edit tags for now
        
        if '?' in line:
            
            # find which bracket and separator form we have for the ? selections
            match = False
            for f, form in enumerate(forms):
                if form.re.search(line):
                    match = True
                    break
            assert match, f'{lnum}: {line}'
            
            l0, l1 = line.index(forms[f].op), line.index(forms[f].cl)
            items = line[l0+1:l1].split(forms[f].sep)
            
            puts = []
            best_y = -float('inf')
            for item in items:
                assert '?' in item, f'item={repr(item)} | {lnum}: {line}'
                item = item.split('?')
                put, x = item[0], float(item[1])
                y = samp(x)
                if y > best_y:
                    best_put = put
                if y > 0:
                    puts.append(put)
            if not puts: # if nothing is sampled, add in the one with highest draw
                puts.append(best_put)
            
            yield line[:l0+1] + forms[f].sep.join(puts) + line[l1:]
        else:
            yield line

In [479]:
def make_fsamp(seed):
    def fsamp(x):
        return x - fsamp.r.random()
    fsamp.r = random.Random(seed)
    return fsamp

def gen_features(i):
    fsamp = make_fsamp(i)
    while True:
        q = '\n'.join(fill_template(fffTemplate.splitlines(), fsamp, None))
        f = list(filter(FFF.query(q), feats))
        if not (11 <= len(f) < 159):
            continue
        return f, q

# Scratcher

In [488]:
# queries = ['\n'.join(fill_template(fffTemplate.splitlines(), fsamp, None)) for i in range(100)]

In [467]:
# i = 0

In [489]:
# i += 10
# fsamp.r = random.Random(i)
# queries = ['\n'.join(fill_template(fffTemplate.splitlines(), fsamp, None)) for i in range(1)]
# q = queries[0]
# guy = list(filter(FFF.query(q), feats))
# print(len(guy))
# print(q)

In [490]:
# flist = [[print_progress(i), set(filter(FFF.query(q), feats))][1] for i,q in enumerate(queries)]

In [491]:
# for q, l in zip(queries, map(len,flist)):
#     print('='*90)
#     print(l)
#     print(q)